### **PROCESO DE ETL PARA EL DATASET `movies_dataset.csv`**

In [6]:
import pandas as pd

##### **CARGA DE LOS DATOS**

In [7]:
movies_df = pd.read_csv('C:\DYNAMO\DATA SCIENCE\Labs-MLOps\Dataset\movies_dataset.csv')
movies_df.head(3)

C:\Users\marce\AppData\Local\Temp\ipykernel_37292\259364485.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv('C:\DYNAMO\DATA SCIENCE\Labs-MLOps\Dataset\movies_dataset.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


#### INSEPCCIÓN INICIAL

In [10]:
movies_df.shape

(45466, 24)

In [8]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [9]:
movies_df.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

##### **DESANIDAR CAMPOS ANIDADOS**

Revisaremos el tipo de dato de cada columna anidada antes de aplicar la transformación ya que pueden venir como diccionarios o en cadena de texto. Teniendo esto presente, el proceso para desanidarlos en cuyo caso será diferente.

También manejaremos los valores null para evitar problemas en el desanidado.

In [11]:
# Verificamos el type de las columnas anidadas
anidadas = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']

for columna in anidadas:
    print(f"Columna: {columna}")
    print(movies_df[columna].apply(type).value_counts())
    print("\n")


Columna: belongs_to_collection
belongs_to_collection
<class 'float'>    40972
<class 'str'>       4494
Name: count, dtype: int64


Columna: genres
genres
<class 'str'>    45466
Name: count, dtype: int64


Columna: production_companies
production_companies
<class 'str'>      45463
<class 'float'>        3
Name: count, dtype: int64


Columna: production_countries
production_countries
<class 'str'>      45463
<class 'float'>        3
Name: count, dtype: int64


Columna: spoken_languages
spoken_languages
<class 'str'>      45460
<class 'float'>        6
Name: count, dtype: int64




#### Manejo de nulls y desanidado de `'belongs_to_collection'`

Hay ciertos valores que NO serán útiles, podría modificar el código para desanidar los campos relevantes desde el principio, evitando así la creación de columnas innecesarias.

Sin embargo, voy a optar por un desanidado completo para tener la flexibilidad de poder revisar todo por si acaso, luego eliminaré columnas innecesarias 

In [12]:
import ast

def convertir_a_diccionario(df: pd.DataFrame, columna: str) -> pd.DataFrame:
    """
    Convierte cadenas en formato de diccionarios o listas en objetos Python reales usando ast.literal_eval.
    
    Esta función toma una columna de un DataFrame y convierte cualquier valor que sea una cadena en un diccionario
    utilizando la función ast.literal_eval, que evalúa las cadenas como expresiones literales de Python.
    Si el valor es nulo (NaN), lo deja como está o lo convierte en un diccionario vacío.
    
    Parámetros:
    ----------
    df : pandas.DataFrame
        El DataFrame que contiene la columna que será procesada.
    
    columna : str
        El nombre de la columna en la que se aplicará la conversión.
    
    Retorna:
    -------
    pandas.DataFrame
        El DataFrame original con la columna especificada convertida a diccionarios.
    
    Ejemplo de uso:
    ---------------
    df = convertir_a_diccionario(df, 'belongs_to_collection')
    """
    df[columna] = df[columna].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else ({} if pd.isnull(x) else x)
    )
    return df

In [13]:
# Se aplica la función a la columna 'belongs_to_collection' y se verifica el resultado
movies_df = convertir_a_diccionario(movies_df, 'belongs_to_collection')
print(movies_df['belongs_to_collection'].head(20))

0     {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                    {}
2     {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                    {}
4     {'id': 96871, 'name': 'Father of the Bride Col...
5                                                    {}
6                                                    {}
7                                                    {}
8                                                    {}
9     {'id': 645, 'name': 'James Bond Collection', '...
10                                                   {}
11                                                   {}
12    {'id': 117693, 'name': 'Balto Collection', 'po...
13                                                   {}
14                                                   {}
15                                                   {}
16                                                   {}
17                                              

In [14]:
def desanidar_columnas(df: pd.DataFrame, columna: str, prefijo: str) -> pd.DataFrame:
    """
    Desanida una columna que contiene diccionarios y extrae sus claves como nuevas columnas.
    
    Parámetros:
    df (pd.DataFrame): El DataFrame que contiene los datos.
    columna (str): El nombre de la columna que se va a desanidar.
    prefijo (str): El prefijo específico para las nuevas columnas desanidadas.
    
    Retorna:
    pd.DataFrame: El DataFrame con las nuevas columnas desanidadas.
    """
    # Desanidar la columna que contiene diccionarios
    desanidado = pd.json_normalize(df[columna])
    
    # Renombrar las columnas desanidadas usando el prefijo específico
    desanidado.columns = [f"{prefijo}_{col}" for col in desanidado.columns]
    
    # Concatenar las nuevas columnas al DataFrame original
    df = pd.concat([df, desanidado], axis=1)
    
    return df

In [15]:
# Desanidar la columna 'belongs_to_collection' con prefijo 'collection'
movies_df = desanidar_columnas(movies_df, 'belongs_to_collection', prefijo='collection')

In [16]:
# Verificar las nuevas columnas desanidadas
print(movies_df[['collection_id', 'collection_name', 'collection_poster_path']].head())

   collection_id                 collection_name  \
0        10194.0            Toy Story Collection   
1            NaN                             NaN   
2       119050.0       Grumpy Old Men Collection   
3            NaN                             NaN   
4        96871.0  Father of the Bride Collection   

             collection_poster_path  
0  /7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg  
1                               NaN  
2  /nLvUdqgPgm3F85NMCii9gVFUcet.jpg  
3                               NaN  
4  /nts4iOmNnq7GNicycMJ9pSAn204.jpg  


In [17]:
movies_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,collection_id,collection_name,collection_poster_path,collection_backdrop_path
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,False,{},65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,NaN,NaN,NaN
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,False,{},16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,NaN,NaN,NaN,NaN
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg


#### Desanidado de `'genres'`
Los valores son listas de dicts, por lo tanto, la función `'desanidar_columnas'` no va a funcionar. Lo manejaré de manera diferente

In [18]:
# Obtenemos una muestra de los valores en la columna 'genres'
print(movies_df['genres'].head(10))

0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
5    [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...
6    [{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...
7    [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
8    [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
9    [{'id': 12, 'name': 'Adventure'}, {'id': 28, '...
Name: genres, dtype: object


In [19]:
import json
import re

def limpiar_json_invalido(texto: str) -> list:
    """
    Corrige un texto JSON que utiliza comillas simples en lugar de comillas dobles
    y lo convierte en un objeto Python. Si el texto no es válido como JSON, retorna una lista vacía.
    
    Parámetros:
    texto (str): El texto que representa un JSON potencialmente inválido.
    
    Retorna:
    list: Una lista con los datos del JSON corregido o una lista vacía en caso de error.
    """
    if isinstance(texto, str):
        # Reemplazar comillas simples por comillas dobles
        texto_limpio = re.sub(r"'", '"', texto)
        try:
            # Intentar cargar el JSON limpio
            return json.loads(texto_limpio)
        except json.JSONDecodeError:
            # Si sigue habiendo un error, devolver una lista vacía
            return []
    return texto


In [20]:
# Se Aplica la limpieza a la columna 'genres' y se verificar la conversión
movies_df['genres'] = movies_df['genres'].apply(limpiar_json_invalido)
print(movies_df['genres'].head(10))


0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
5    [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...
6    [{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...
7    [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
8    [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
9    [{'id': 12, 'name': 'Adventure'}, {'id': 28, '...
Name: genres, dtype: object


In [21]:
def desanidar_listas_diccionarios(df: pd.DataFrame, columna: str, prefijo: str) -> pd.DataFrame:
    """
    Desanida una columna que contiene listas de diccionarios, explota cada elemento en una fila separada
    y extrae las claves de los diccionarios como nuevas columnas en el DataFrame.
    
    Parámetros:
    df (pd.DataFrame): El DataFrame que contiene los datos.
    columna (str): El nombre de la columna que se va a desanidar.
    prefijo (str): El prefijo para las nuevas columnas desanidadas.
    
    Retorna:
    pd.DataFrame: El DataFrame con las nuevas columnas desanidadas y concatenadas.
    """
    # Explotar la lista de diccionarios para convertir cada elemento en una fila separada
    df = df.explode(columna).reset_index(drop=True)
    
    # Desanidar las claves de los diccionarios en columnas separadas
    desanidado = pd.json_normalize(df[columna])
    desanidado.columns = [f"{prefijo}_{col}" for col in desanidado.columns]
    
    # Concatenar las nuevas columnas desanidadas al DataFrame original
    df = pd.concat([df, desanidado], axis=1)
    
    return df


In [22]:
# Se aplica la función de desanidado y se verifica el resultado
movies_df = desanidar_listas_diccionarios(movies_df, 'genres', 'genre')
print(movies_df[['genre_id', 'genre_name']].head(20))

    genre_id genre_name
0       16.0  Animation
1       35.0     Comedy
2    10751.0     Family
3       12.0  Adventure
4       14.0    Fantasy
5    10751.0     Family
6    10749.0    Romance
7       35.0     Comedy
8       35.0     Comedy
9       18.0      Drama
10   10749.0    Romance
11      35.0     Comedy
12      28.0     Action
13      80.0      Crime
14      18.0      Drama
15      53.0   Thriller
16      35.0     Comedy
17   10749.0    Romance
18      28.0     Action
19      12.0  Adventure


#### Desanidado de `'product_companies'`

In [23]:
# Obtenemos una muestra de los valores en la columna 'production_companies'
print(movies_df['production_companies'].head(10))

0       [{'name': 'Pixar Animation Studios', 'id': 3}]
1       [{'name': 'Pixar Animation Studios', 'id': 3}]
2       [{'name': 'Pixar Animation Studios', 'id': 3}]
3    [{'name': 'TriStar Pictures', 'id': 559}, {'na...
4    [{'name': 'TriStar Pictures', 'id': 559}, {'na...
5    [{'name': 'TriStar Pictures', 'id': 559}, {'na...
6    [{'name': 'Warner Bros.', 'id': 6194}, {'name'...
7    [{'name': 'Warner Bros.', 'id': 6194}, {'name'...
8    [{'name': 'Twentieth Century Fox Film Corporat...
9    [{'name': 'Twentieth Century Fox Film Corporat...
Name: production_companies, dtype: object


In [24]:
movies_df['production_companies'].apply(type).value_counts()

production_companies
<class 'str'>      93539
<class 'float'>        9
Name: count, dtype: int64

In [25]:
# Se aplica la limpieza a la columna 'production_companies'
movies_df['production_companies'] = movies_df['production_companies'].apply(limpiar_json_invalido)

In [26]:
# Se aplica la función de desanidado en 'production_companies' con el prefijo 'company'
movies_df = desanidar_listas_diccionarios(movies_df, 'production_companies', 'company')

# Se verifican las nuevas columnas desanidadas
print(movies_df[['company_id', 'company_name']].head(20))


    company_id                            company_name
0          3.0                 Pixar Animation Studios
1          3.0                 Pixar Animation Studios
2          3.0                 Pixar Animation Studios
3        559.0                        TriStar Pictures
4       2550.0                            Teitler Film
5      10201.0               Interscope Communications
6        559.0                        TriStar Pictures
7       2550.0                            Teitler Film
8      10201.0               Interscope Communications
9        559.0                        TriStar Pictures
10      2550.0                            Teitler Film
11     10201.0               Interscope Communications
12      6194.0                            Warner Bros.
13     19464.0                          Lancaster Gate
14      6194.0                            Warner Bros.
15     19464.0                          Lancaster Gate
16       306.0  Twentieth Century Fox Film Corporation
17       3

In [27]:
# Convertir los valores de 'company_id' a enteros
movies_df['company_id'] = movies_df['company_id'].astype('Int64')

# Verificamos nuevamente las primeras filas
print(movies_df[['company_id', 'company_name']].head(20))


    company_id                            company_name
0            3                 Pixar Animation Studios
1            3                 Pixar Animation Studios
2            3                 Pixar Animation Studios
3          559                        TriStar Pictures
4         2550                            Teitler Film
5        10201               Interscope Communications
6          559                        TriStar Pictures
7         2550                            Teitler Film
8        10201               Interscope Communications
9          559                        TriStar Pictures
10        2550                            Teitler Film
11       10201               Interscope Communications
12        6194                            Warner Bros.
13       19464                          Lancaster Gate
14        6194                            Warner Bros.
15       19464                          Lancaster Gate
16         306  Twentieth Century Fox Film Corporation
17        

In [28]:
# Continuamente observamos el dataframe
movies_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,vote_average,vote_count,collection_id,collection_name,collection_poster_path,collection_backdrop_path,genre_id,genre_name,company_name,company_id
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"{'id': 16, 'name': 'Animation'}",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,7.7,5415.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,16.0,Animation,Pixar Animation Studios,3
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"{'id': 35, 'name': 'Comedy'}",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,7.7,5415.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,35.0,Comedy,Pixar Animation Studios,3
2,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"{'id': 10751, 'name': 'Family'}",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,7.7,5415.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,10751.0,Family,Pixar Animation Studios,3
3,False,{},65000000,"{'id': 12, 'name': 'Adventure'}",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,6.9,2413.0,NaN,NaN,NaN,NaN,12.0,Adventure,TriStar Pictures,559
4,False,{},65000000,"{'id': 12, 'name': 'Adventure'}",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,6.9,2413.0,NaN,NaN,NaN,NaN,12.0,Adventure,Teitler Film,2550


#### Desanidado de `'production_countries'`

In [29]:
# Obtener una muestra de los valores en la columna 'production_countries'
print(movies_df['production_countries'].head(10))

0    [{'iso_3166_1': 'US', 'name': 'United States o...
1    [{'iso_3166_1': 'US', 'name': 'United States o...
2    [{'iso_3166_1': 'US', 'name': 'United States o...
3    [{'iso_3166_1': 'US', 'name': 'United States o...
4    [{'iso_3166_1': 'US', 'name': 'United States o...
5    [{'iso_3166_1': 'US', 'name': 'United States o...
6    [{'iso_3166_1': 'US', 'name': 'United States o...
7    [{'iso_3166_1': 'US', 'name': 'United States o...
8    [{'iso_3166_1': 'US', 'name': 'United States o...
9    [{'iso_3166_1': 'US', 'name': 'United States o...
Name: production_countries, dtype: object


In [30]:
# Se aplica la limpieza a la columna 'production_countries'
movies_df['production_countries'] = movies_df['production_countries'].apply(limpiar_json_invalido)

In [31]:
# Desanidamos la columna 'production_countries' con el prefijo 'country'
movies_df = desanidar_listas_diccionarios(movies_df, 'production_countries', 'country')

# Verificamos las nuevas columnas desanidadas
print(movies_df[['country_iso_3166_1', 'country_name']].head(20))


   country_iso_3166_1              country_name
0                  US  United States of America
1                  US  United States of America
2                  US  United States of America
3                  US  United States of America
4                  US  United States of America
5                  US  United States of America
6                  US  United States of America
7                  US  United States of America
8                  US  United States of America
9                  US  United States of America
10                 US  United States of America
11                 US  United States of America
12                 US  United States of America
13                 US  United States of America
14                 US  United States of America
15                 US  United States of America
16                 US  United States of America
17                 US  United States of America
18                 US  United States of America
19                 US  United States of 

In [32]:
movies_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,collection_id,collection_name,collection_poster_path,collection_backdrop_path,genre_id,genre_name,company_name,company_id,country_iso_3166_1,country_name
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"{'id': 16, 'name': 'Animation'}",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,16.0,Animation,Pixar Animation Studios,3,US,United States of America
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"{'id': 35, 'name': 'Comedy'}",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,35.0,Comedy,Pixar Animation Studios,3,US,United States of America
2,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"{'id': 10751, 'name': 'Family'}",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,10751.0,Family,Pixar Animation Studios,3,US,United States of America
3,False,{},65000000,"{'id': 12, 'name': 'Adventure'}",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,NaN,NaN,NaN,NaN,12.0,Adventure,TriStar Pictures,559,US,United States of America
4,False,{},65000000,"{'id': 12, 'name': 'Adventure'}",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,NaN,NaN,NaN,NaN,12.0,Adventure,Teitler Film,2550,US,United States of America


#### Desanidado de `'spoken_lenguages'`

In [33]:
# Muestra de los valores en la columna 'spoken_languages'
print(movies_df['spoken_languages'].head(10))


0             [{'iso_639_1': 'en', 'name': 'English'}]
1             [{'iso_639_1': 'en', 'name': 'English'}]
2             [{'iso_639_1': 'en', 'name': 'English'}]
3    [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
4    [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
5    [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
6    [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
7    [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
8    [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
9    [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
Name: spoken_languages, dtype: object


In [34]:
# Se aplica la limpieza JSON a la columna 'spoken_languages'
movies_df['spoken_languages'] = movies_df['spoken_languages'].apply(limpiar_json_invalido)

In [35]:
# Se aplica la función de desanidado en 'spoken_languages'
movies_df = desanidar_listas_diccionarios(movies_df, 'spoken_languages', 'language')

In [36]:
# Corrobarmos las nuevas columnas desanidadas
print(movies_df[['language_iso_639_1', 'language_name']].head(20))

   language_iso_639_1 language_name
0                  en       English
1                  en       English
2                  en       English
3                  en       English
4                  fr      Français
5                  en       English
6                  fr      Français
7                  en       English
8                  fr      Français
9                  en       English
10                 fr      Français
11                 en       English
12                 fr      Français
13                 en       English
14                 fr      Français
15                 en       English
16                 fr      Français
17                 en       English
18                 fr      Français
19                 en       English


In [37]:
movies_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,collection_poster_path,collection_backdrop_path,genre_id,genre_name,company_name,company_id,country_iso_3166_1,country_name,language_iso_639_1,language_name
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"{'id': 16, 'name': 'Animation'}",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,16.0,Animation,Pixar Animation Studios,3,US,United States of America,en,English
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"{'id': 35, 'name': 'Comedy'}",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,35.0,Comedy,Pixar Animation Studios,3,US,United States of America,en,English
2,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"{'id': 10751, 'name': 'Family'}",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,10751.0,Family,Pixar Animation Studios,3,US,United States of America,en,English
3,False,{},65000000,"{'id': 12, 'name': 'Adventure'}",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,NaN,NaN,12.0,Adventure,TriStar Pictures,559,US,United States of America,en,English
4,False,{},65000000,"{'id': 12, 'name': 'Adventure'}",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,NaN,NaN,12.0,Adventure,TriStar Pictures,559,US,United States of America,fr,Français


##### **IMPUTACIÓN DE VALORES Y COLUMNAS Y TRATAMIENTO DE FECHAS**

Se reemplazarán los valores nulos de las columnas requeridas, como así también, se eliminarán las columnas que no serán de utilidad para nuestro modelo:
- columnas anidadas (trabajaremos con las desanidadas)
- columnas relacionadas con URL e imágenes que no nos aportarán al análisis.
- columnas con texto que describen la película (no utilizaremos procesamiento de lenguaje natural, pero podría considerarse para perfeccionar el modelo).

Por último, se implementa el siguiente formato de fechas: `AAAA-mm-dd`



In [38]:
import pandas as pd

In [39]:
movies_df.shape

(381049, 36)

In [40]:
movies_df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'collection_id', 'collection_name',
       'collection_poster_path', 'collection_backdrop_path', 'genre_id',
       'genre_name', 'company_name', 'company_id', 'country_iso_3166_1',
       'country_name', 'language_iso_639_1', 'language_name'],
      dtype='object')

In [41]:
# Eliminamos las columnas anidadas
columns_to_drop = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']
movies_df = movies_df.drop(columns=columns_to_drop)

In [42]:
print(movies_df.columns)

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'collection_id',
       'collection_name', 'collection_poster_path', 'collection_backdrop_path',
       'genre_id', 'genre_name', 'company_name', 'company_id',
       'country_iso_3166_1', 'country_name', 'language_iso_639_1',
       'language_name'],
      dtype='object')


In [43]:
# Eliminamos las columnas relacionadas con imágenes
columns_to_drop = ['poster_path', 'collection_poster_path', 'collection_backdrop_path']
movies_df = movies_df.drop(columns=columns_to_drop)

In [44]:
print(movies_df.columns)

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'release_date', 'revenue',
       'runtime', 'status', 'tagline', 'title', 'video', 'vote_average',
       'vote_count', 'collection_id', 'collection_name', 'genre_id',
       'genre_name', 'company_name', 'company_id', 'country_iso_3166_1',
       'country_name', 'language_iso_639_1', 'language_name'],
      dtype='object')


In [45]:
# Eliminamos columnas que no serán útiles
columns_to_drop = ['adult', 'original_title', 'original_title', 'homepage', 'imdb_id', 'tagline', 'overview', 'status', 'video']
movies_df = movies_df.drop(columns=columns_to_drop)

In [46]:
movies_df.shape

(381049, 20)

In [47]:
movies_df.columns

Index(['budget', 'id', 'original_language', 'popularity', 'release_date',
       'revenue', 'runtime', 'title', 'vote_average', 'vote_count',
       'collection_id', 'collection_name', 'genre_id', 'genre_name',
       'company_name', 'company_id', 'country_iso_3166_1', 'country_name',
       'language_iso_639_1', 'language_name'],
      dtype='object')

MANEJO DE VALORES NULOS DE LAS COLUMNAS SOLICITADAS:
`revenue` - `budget` - `release_date`

In [48]:
# Se reemplazan los nulos en 'revenue' y 'budget' con 0
movies_df['revenue'].fillna(0, inplace=True)
movies_df['budget'].fillna(0, inplace=True)


C:\Users\marce\AppData\Local\Temp\ipykernel_37292\2088379519.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies_df['revenue'].fillna(0, inplace=True)
C:\Users\marce\AppData\Local\Temp\ipykernel_37292\2088379519.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

In [49]:
movies_df.isnull().sum()

budget                     0
id                         0
original_language         96
popularity                17
release_date             179
revenue                    0
runtime                  577
title                     21
vote_average              21
vote_count                21
collection_id         327816
collection_name       327816
genre_id                3184
genre_name              3184
company_name           29699
company_id             29699
country_iso_3166_1     11456
country_name           11456
language_iso_639_1      8085
language_name           8085
dtype: int64

In [50]:
# Se eliminan las filas con valores nulos en 'release_date'
movies_df = movies_df.dropna(subset=['release_date'])
print(movies_df['release_date'].isnull().sum())

0


MANEJO DE FORMATO DE FECHAS

In [51]:
# Asegurar que la columna release_date esté en formato de fecha
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], format='%Y-%m-%d', errors='coerce')

In [52]:
movies_df['release_date'].head(10)

0   1995-10-30
1   1995-10-30
2   1995-10-30
3   1995-12-15
4   1995-12-15
5   1995-12-15
6   1995-12-15
7   1995-12-15
8   1995-12-15
9   1995-12-15
Name: release_date, dtype: datetime64[ns]

In [53]:
# Se crea la nueva columna release_year extrayendo el año de release_date
movies_df['release_year'] = movies_df['release_date'].dt.year

In [54]:
# Corroboramos los cambios implementados
print(movies_df[['release_date', 'release_year']].head(10))

  release_date  release_year
0   1995-10-30        1995.0
1   1995-10-30        1995.0
2   1995-10-30        1995.0
3   1995-12-15        1995.0
4   1995-12-15        1995.0
5   1995-12-15        1995.0
6   1995-12-15        1995.0
7   1995-12-15        1995.0
8   1995-12-15        1995.0
9   1995-12-15        1995.0


In [55]:
# Se convierte 'release_year' a entero y se verifica el cambio
movies_df['release_year'] = movies_df['release_year'].astype('Int64')
print(movies_df[['release_year']].dtypes)


release_year    Int64
dtype: object


In [56]:
movies_df['release_year'].head()

0    1995
1    1995
2    1995
3    1995
4    1995
Name: release_year, dtype: Int64

CREAMOS LA COLUMNA `return` 

La misma contiene el retorno de inversión.

Se divide `revenue` entre `budget` solo si ambos valores son mayores que 0 y se asigna el valor 0 en caso de que no haya datos disponibles o uno de los valores sea 0. Tal como se lo solicita.

In [57]:
# Nos fijamos que las columnas 'revenue' y 'budget' esten en el formato correcto
movies_df['revenue'] = pd.to_numeric(movies_df['revenue'], errors='coerce')
movies_df['budget'] = pd.to_numeric(movies_df['budget'], errors='coerce')

In [58]:
# se verifican los tipos de datos y valores únicos en las columnas 'budget' y 'revenue'
print(movies_df['budget'].dtype)
print(movies_df['revenue'].dtype)

float64
float64


In [59]:
# Revisamos si hay valores no numéricos
print(movies_df['budget'].unique()[:20])  
print("\n")
print(movies_df['revenue'].unique()[:20])  


[30000000. 65000000.        0. 16000000. 60000000. 58000000. 35000000.
 62000000. 44000000. 98000000. 52000000. 16500000.  4000000. 30250000.
 50000000.  3600000. 12000000. 18000000. 29500000. 11000000.]


[3.73554033e+08 2.62797249e+08 0.00000000e+00 8.14521560e+07
 7.65789110e+07 1.87436818e+08 6.43501710e+07 3.52194034e+08
 1.07879496e+08 1.13483240e+07 1.36817650e+07 1.00173220e+07
 1.16112375e+08 1.35000000e+08 4.30000000e+06 2.12385533e+08
 3.54311130e+07 1.15101622e+08 3.03030720e+07 4.98000000e+07]


In [60]:
# Se crea la columna 'return' dividiendo 'revenue' entre 'budget', asignando 0 si no es posible calcularlo
movies_df['return'] = movies_df.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] > 0 and row['revenue'] > 0 else 0, axis=1)

In [61]:
# Miramos los primeros valores
print(movies_df[['budget', 'revenue', 'return']].head(10))

       budget      revenue     return
0  30000000.0  373554033.0  12.451801
1  30000000.0  373554033.0  12.451801
2  30000000.0  373554033.0  12.451801
3  65000000.0  262797249.0   4.043035
4  65000000.0  262797249.0   4.043035
5  65000000.0  262797249.0   4.043035
6  65000000.0  262797249.0   4.043035
7  65000000.0  262797249.0   4.043035
8  65000000.0  262797249.0   4.043035
9  65000000.0  262797249.0   4.043035


In [62]:
movies_df['popularity'].head()

0    21.946943
1    21.946943
2    21.946943
3    17.015539
4    17.015539
Name: popularity, dtype: object

In [63]:
movies_df.shape

(380870, 22)

##### **EXPORTACIÓN EN FORMATO PARQUET**

El dataframe finalmente procesado será convertido en Parquet, debido a que este formato es más eficiente para cargas más rápidas.

In [64]:
# Se convirtió la columna 'popularity' a valores numéricos
movies_df['popularity'] = pd.to_numeric(movies_df['popularity'], errors='coerce')
print(movies_df['popularity'].head())

0    21.946943
1    21.946943
2    21.946943
3    17.015539
4    17.015539
Name: popularity, dtype: float64


In [65]:
movies_df.to_parquet('movies_etl_final.parquet', index=False)

In [66]:
import pandas as pd

In [67]:
df = pd.read_parquet('movies_etl_final.parquet')

In [68]:
df.shape

(380870, 22)

### **PROCESO DE ETL PARA EL DATASET `credits.csv`**

Se convierte el dataset en formato parquet para un manejo más eficiente

In [69]:
import pandas as pd

In [70]:
# credits_df = pd.read_csv('C:\DYNAMO\DATA SCIENCE\Labs-MLOps\Dataset\credits.csv')

In [72]:
# Convertimos el DataFrame en formato parquet
## credits_df.to_parquet('credits.parquet', index=False)

In [73]:
# Leemos el archivo Parquet en lugar del CSV
credits_df = pd.read_parquet('credits.parquet')

In [74]:
credits_df.shape

(45476, 3)

#### INSPECCIÓN INICIAL

In [75]:
credits_df.head(10)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
5,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",949
6,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...",11860
7,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",45325
8,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",9091
9,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",710


In [76]:
# Verificar el contenido de la columna 'cast'
print(credits_df['cast'].iloc[0])

[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4tN

##### **PROCESO DE DESANIDADO**

#### Desanidado de `'cast'` 
Aquí procederemos a desanidar solo aquellos valores que nos serán útiles desde el inicio, descartando todo lo demás.

In [77]:
import ast

def desanidar_cast(df: pd.DataFrame) -> pd.DataFrame:
    """
    Esta función desanida la columna 'cast' de un DataFrame que contiene información sobre los actores y los personajes que interpretan en formato de lista de diccionarios. 
    El resultado es un DataFrame donde cada fila representa un actor y su respectivo personaje para una película.

    Parámetros:
    -----------
    df : pd.DataFrame
        El DataFrame que contiene la columna 'cast' en formato de cadena de texto (JSON) o como lista de diccionarios. 
        Cada diccionario en 'cast' debe tener las claves 'name' (nombre del actor) y 'character' (personaje que interpreta).

    Retorno:
    --------
    pd.DataFrame
        DataFrame con las siguientes columnas:
        - 'actor_name': Nombre del actor.
        - 'character': Personaje interpretado por el actor.
    
    El DataFrame original es modificado, eliminando la columna 'cast' y explotando la lista de actores para que cada actor ocupe una fila separada.
    
    Notas:
    ------
    - La columna 'cast' es desanidada y convertida en dos columnas: 'actor_name' (nombre del actor) y 'character' (personaje interpretado).
    - Si hay valores nulos en 'cast', se eliminan las filas correspondientes.
    - Si la columna 'cast' está en formato de cadena de texto (JSON), esta es convertida a una lista de diccionarios antes de desanidarse.
    """
    # Verificar si la columna es una cadena de texto (string) y convertirla en lista de diccionarios
    df['cast'] = df['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

    # Asegurarnos de eliminar valores nulos
    df = df.dropna(subset=['cast'])

    # Desanidar los valores de la columna 'cast' (lista de diccionarios)
    df['actor_character'] = df['cast'].apply(lambda x: [(actor.get('name'), actor.get('character')) for actor in x if isinstance(actor, dict)])

    # Explode para separar las listas en filas individuales
    df = df.explode('actor_character')

    # Dividir 'actor_character' en 'actor_name' y 'character', asegurándonos de manejar valores nulos
    df['actor_name'] = df['actor_character'].apply(lambda x: x[0] if isinstance(x, tuple) else None)
    df['character'] = df['actor_character'].apply(lambda x: x[1] if isinstance(x, tuple) else None)

    # Eliminar la columna 'actor_character' ya que no es necesaria después de la división
    df = df.drop(columns=['actor_character'])

    return df


In [78]:
# se aplica la función para desanidar la columna 'cast'
credits_df = desanidar_cast(credits_df)

In [80]:
# se verifica el resultado
print(credits_df[['actor_name', 'character']].head(10))

          actor_name                character
0          Tom Hanks            Woody (voice)
0          Tim Allen   Buzz Lightyear (voice)
0        Don Rickles  Mr. Potato Head (voice)
0         Jim Varney       Slinky Dog (voice)
0      Wallace Shawn              Rex (voice)
0  John Ratzenberger             Hamm (voice)
0        Annie Potts          Bo Peep (voice)
0        John Morris             Andy (voice)
0    Erik von Detten              Sid (voice)
0     Laurie Metcalf       Mrs. Davis (voice)


In [81]:
credits_df.shape

(564892, 5)

In [82]:
# Verificar el contenido de la columna 'crew'
print(credits_df['crew'].iloc[0])

[{'credit_id': '52fe4284c3a36847f8024f49', 'department': 'Directing', 'gender': 2, 'id': 7879, 'job': 'Director', 'name': 'John Lasseter', 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}, {'credit_id': '52fe4284c3a36847f8024f4f', 'department': 'Writing', 'gender': 2, 'id': 12891, 'job': 'Screenplay', 'name': 'Joss Whedon', 'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'}, {'credit_id': '52fe4284c3a36847f8024f55', 'department': 'Writing', 'gender': 2, 'id': 7, 'job': 'Screenplay', 'name': 'Andrew Stanton', 'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'}, {'credit_id': '52fe4284c3a36847f8024f5b', 'department': 'Writing', 'gender': 2, 'id': 12892, 'job': 'Screenplay', 'name': 'Joel Cohen', 'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'}, {'credit_id': '52fe4284c3a36847f8024f61', 'department': 'Writing', 'gender': 0, 'id': 12893, 'job': 'Screenplay', 'name': 'Alec Sokolow', 'profile_path': '/v79vlRYi94BZUQnkkyznbGUZLjT.jpg'}, {'credit_id': '52fe4284c3a36847f8024f67', 'depart

#### **Columna `'crew'`:**
He tenido diversos problemas durante el proceso de desanidado de esta columna. Todos los procedimientos que he implementado han resultado ser ineficientes, presentando problemas de rendimiento llegando a sobrepasar los 10min. durante la ejecución del script. 

Luego de intentar optimizar reitaradas veces las funciones que han sido implementadas y sin obtener un resultado óptimo, tomaré la decisión de finalizar aquí el proceso de ETL, descartando por completo los valores de la columna 'crew' siendo así, imposible de implementar el endpoint solicitado `def get_director(nombre_director):`

##### **ELIMINAR COLUMNAS INNECESARIAS**

In [83]:
credits_df.columns

Index(['cast', 'crew', 'id', 'actor_name', 'character'], dtype='object')

In [84]:
# Eliminamos las columnas anidadas
columns_to_drop = ['cast', 'crew']
credits_df = credits_df.drop(columns=columns_to_drop)

In [86]:
credits_df.columns

Index(['id', 'actor_name', 'character'], dtype='object')

In [87]:
credits_df.shape

(564892, 3)

EXPORTACIÓN EN FORMATO PARQUET

In [88]:
credits_df.to_parquet('credits_etl_final.parquet', index=False)

##### **JOIN DE DATAFRAMES**

##### Uniremos los dataframes ya procesados para mejorar la optimización y rendimiento de la API
Para ello cargamos `movies_etl_final.parquet` que contiene el proceso ETL final de `movies_dataset.csv`

También cargamos `credits_etl_final.parquet` que contiene los datos desanidados de `credits.csv`

In [89]:
import pandas as pd

# Cargamos los archivos
movies_df = pd.read_parquet('movies_etl_final.parquet')
credits_df = pd.read_parquet('credits_etl_final.parquet')

In [91]:
movies_df.shape

(380870, 22)

In [92]:
credits_df.shape

(564892, 3)

In [90]:
# Verificamos las columnas comunes entre ambos DataFrames
print(movies_df.columns)
print("\n")
print(credits_df.columns)

Index(['budget', 'id', 'original_language', 'popularity', 'release_date',
       'revenue', 'runtime', 'title', 'vote_average', 'vote_count',
       'collection_id', 'collection_name', 'genre_id', 'genre_name',
       'company_name', 'company_id', 'country_iso_3166_1', 'country_name',
       'language_iso_639_1', 'language_name', 'release_year', 'return'],
      dtype='object')


Index(['id', 'actor_name', 'character'], dtype='object')


#### Haremos el join a través de "id"

In [93]:
# Convertimos las columnas 'id' a numérico
movies_df['id'] = pd.to_numeric(movies_df['id'], errors='coerce')
credits_df['id'] = pd.to_numeric(credits_df['id'], errors='coerce')

In [94]:
# Realizamos la unión de ambos DataFrames usando la columna 'id'
merged_df = pd.merge(movies_df, credits_df, on='id', how='left')

In [95]:
merged_df.head(10)

,budget,id,original_language,popularity,release_date,revenue,runtime,title,vote_average,vote_count,...,company_name,company_id,country_iso_3166_1,country_name,language_iso_639_1,language_name,release_year,return,actor_name,character
0,30000000.0,862.0,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,...,Pixar Animation Studios,3,US,United States of America,en,English,1995,12.451801,Tom Hanks,Woody (voice)
1,30000000.0,862.0,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,...,Pixar Animation Studios,3,US,United States of America,en,English,1995,12.451801,Tim Allen,Buzz Lightyear (voice)
2,30000000.0,862.0,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,...,Pixar Animation Studios,3,US,United States of America,en,English,1995,12.451801,Don Rickles,Mr. Potato Head (voice)
3,30000000.0,862.0,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,...,Pixar Animation Studios,3,US,United States of America,en,English,1995,12.451801,Jim Varney,Slinky Dog (voice)
4,30000000.0,862.0,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,...,Pixar Animation Studios,3,US,United States of America,en,English,1995,12.451801,Wallace Shawn,Rex (voice)
5,30000000.0,862.0,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,...,Pixar Animation Studios,3,US,United States of America,en,English,1995,12.451801,John Ratzenberger,Hamm (voice)
6,30000000.0,862.0,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,...,Pixar Animation Studios,3,US,United States of America,en,English,1995,12.451801,Annie Potts,Bo Peep (voice)
7,30000000.0,862.0,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,...,Pixar Animation Studios,3,US,United States of America,en,English,1995,12.451801,John Morris,Andy (voice)
8,30000000.0,862.0,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,...,Pixar Animation Studios,3,US,United States of America,en,English,1995,12.451801,Erik von Detten,Sid (voice)
9,30000000.0,862.0,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,...,Pixar Animation Studios,3,US,United States of America,en,English,1995,12.451801,Laurie Metcalf,Mrs. Davis (voice)


#### Los valores de 'id' fueron almacenados como floats. Lo solucionamos convirtiendo 'id' a entero.

In [97]:
# Se convierte 'id' a entero después de la unión, eliminando cualquier valor nulo que haya quedado en la columna id
merged_df['id'] = merged_df['id'].fillna(0).astype(int)

In [98]:
# Verificamos la conversión
print(merged_df[['id']].dtypes)

id    int64
dtype: object


In [99]:
merged_df.head(1)

,budget,id,original_language,popularity,release_date,revenue,runtime,title,vote_average,vote_count,...,company_name,company_id,country_iso_3166_1,country_name,language_iso_639_1,language_name,release_year,return,actor_name,character
0,30000000.0,862,en,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7,5415.0,...,Pixar Animation Studios,3,US,United States of America,en,English,1995,12.451801,Tom Hanks,Woody (voice)


In [100]:
merged_df.shape

(6736774, 24)

##### El crecimiento tan grande en el número de filas después del merge sugiere que hay una relación de uno a muchos entre las dos tablas, lo que genera una multiplicación de filas. 

Sucede que una película (representada por su id) tiene múltiples actores en el dataset de credits, por lo que, al hacer el merge, se crea una fila para cada combinación de película y actor, lo que resulta en un aumento exponencial del número de filas.

Para solucionar esto, se tuvo en cuenta el requerimiento solicitado en el endpoint el cual implica contar cuántas películas tiene un actor, lo cual requiere conservar todas las asociaciones actor-película.

- Para ello se concatenan los actores en una sola celda sin duplicar las filas y sin inflar innecesariamente el dataset.

In [101]:
# Contar cuántos actores hay por cada película (id)
actor_count_per_movie = credits_df.groupby('id').size()
print(actor_count_per_movie.describe())


count    45432.000000
mean        12.433791
std         12.046310
min          1.000000
25%          6.000000
50%         10.000000
75%         15.000000
max        313.000000
dtype: float64


Acá se ve que cada película tiene alrededor de 12 actores asociados, con algunas películas que tienen hasta 313 actores. Esto explica la gran multiplicación de filas en el merge.

In [105]:
# Agrupar actores por película y concatenar sus nombres en una sola celda, manejando los valores nulos
credits_df['actor_name'] = credits_df['actor_name'].fillna('')  # Reemplazar nulos por una cadena vacía
credits_df_grouped = credits_df.groupby('id')['actor_name'].apply(lambda x: ', '.join(x)).reset_index()


In [106]:
credits_df_grouped.head()


,id,actor_name
0,2,"Turo Pajala, Susanna Haavisto, Matti Pellonpää..."
1,3,"Matti Pellonpää, Kati Outinen, Sakari Kuosmane..."
2,5,"Tim Roth, Antonio Banderas, Jennifer Beals, Ma..."
3,6,"Emilio Estevez, Cuba Gooding Jr., Denis Leary,..."
4,11,"Mark Hamill, Harrison Ford, Carrie Fisher, Pet..."


In [107]:
# Realizamos la unión de ambos DataFrames usando 'id'
merged_df = pd.merge(movies_df, credits_df_grouped, on='id', how='left')


In [109]:
merged_df.shape

(380870, 23)

#### Para finalizar el proceso, se exportará el df unido y previamente procesado para que la API consuma los datos desde un mismo lugar

In [110]:
merged_df.to_parquet('dataset_final.parquet', index=False)

In [111]:
df = pd.read_parquet('dataset_final.parquet')

In [ ]:
df.shape